In [7]:
import pandas as pd
import numpy as np
import re
import math
import requests
from bs4 import BeautifulSoup
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import porter
nltk.download('punkt')
nltk.download('stopwords')
import matplotlib.pyplot as plt
import psycopg2 as pc

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielstephensen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielstephensen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Function Definitions

In [8]:
#cleantext cleans the input string with the following functions: Characters are set to lowercase, 
#urls are substituted with <URL>, dates are substitured with <DATE>, emails are substitured with <EMAIL>
#numbers are substitured with <NUM>, newlines and non-letter characters are removed.
def cleantext(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'<|>', "", text)
    text = re.sub(r'(https?:\/\/)?w{0,3}\.?[a-z]+\.[a-z]\w*[\w\/-]*', "<URL>", text)
    text = re.sub(r'(jan\.?(uary)?|feb\.?(uary)?|mar\.?(ch)?|apr\.?(il)?|may|jun\.(e)?|jul\.(y)?|aug\.?(ust)?|sep\.?(tember)?|oct\.?(ober)?|nov\.?(ember)?|dec\.?(ember)?|monday|tuesday|wednesday|thursday|friday|saturday|sunday) (the )?\d{1,2}((th)?,?( \d{4})?)?', "<DATE>", text)
    text = re.sub(r'\w+@\w+\.[a-zA-Z]{2,3}', "<EMAIL>", text)
    text = re.sub(r'[0-9]+', "<NUM>", text)
    text = re.sub(r'(\\n)+|\s{2,}|(\\t+)', " ", text)
    text = re.sub(r'\.|,|\\|-|\?|\(|\)|\||&|"|”|“|:|!|\+|-|–|—|\/|\$|%|€|#|;|\[|\]|©|®|…|=', "", text)
    return text

#cleanMetaKeywords cleans the input string with the following functions: 
#Characters are set to lowercase, newlines and non-letter characters are removed.
def cleanMetaKeywords(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'(\\n)+|\s{2,}|(\\t+)', " ", text)
    text = re.sub(r'\.|\\|-|\?|\(|\)|\||&|"|”|“|:|!|\+|-|\'|–|—|\/|\$|%|€|#|;|\[|\]|©|®|…|=|<|>', "", text)
    return text

def tokenize(text):
    return word_tokenize(text)

def stopword(word_list):
    stop_words = stopwords.words('english')
    
    return [word for word in word_list if word not in stop_words]

def stemming(word_list):
    stemmer = porter.PorterStemmer()

    return [stemmer.stem(word) for word in word_list]

def getSoup(url):
    response = requests.get(next_page)
    contents = response.content
    return BeautifulSoup(contents, 'html.parser')



## Task 1

Group members: Angelina Näsström (nzv947), Daniel Stephensen (fbp131), Kristina Wilke (mlt790), Lauritz Koch (hdg618)

## Task 2

We have used the following procedures: cleaning, tokenizing, removing stopwords and stemming the data. When cleaning the data we made sure of the following: 
1. all letters are in lowercase
2. all urls are written as < URL >
3. all dates are written as < DATE >
4. all emails are written as < EMAIL >
5. all numbers are written as < NUM >
6. all unimportant symbols are removed

Converting all letters to lowercase makes it easier to compare different words. Point 2-5 are useful because it makes it possible to count the number of urls, dates, emails and numbers. Also, removing these makes sure that they are not treated as words. Removing unimportant symbols makes sure that these are not treated as words. 

Tokenization makes processing of the data easier, as it eliminates blank spaces and punctuations etc, making the text more homogeneous. In the tokenization process, we, for example, made all the data lower-case, thus not having two different results when processing 'Hello' and 'hello'.

Removing stopwords is useful because these words do not help giving meaning to the documents, in other words they are noise.

Stemming the data is useful because it makes sure that different variants of the same word is converted into the rood of the word. This way it is possible to make sure that two different words (same word with different endings) are understood the same way, because they actually have the exact same meaning.

Implementing task 2 we have used the Pandas library, nltk library and re library. The Pandas library has just been used to read the data from the 'news_sample.csv' file. word_tokenize is a sublibrary of nltk that has some useful functions for tokenizing. stopwords is a sublibrary of nltk.corpus that has some useful functions for removing stopwords. porter is a sublibrary of nltk.stem that has some useful functions for stemming data. These three sublibraries are useful because you do not need to create your own complex functions to tokenize, remove stopwords and stem the data. We have not used the clean_text library because we it did not have all the functionality needed for the task. 

In [9]:
#Reading the data from news_sample.csv
datasample = pd.read_csv('news_sample.csv')

In [10]:
#Cleaning the content
cleaned_content = [cleantext(article_content) for article_content in datasample['content']]

#Tokenizing the cleaned data
tokens = [tokenize(clean_text) for clean_text in cleaned_content]

#Removing stopwords
stopwords = [stopword(token_list) for token_list in tokens]

#Stemming the data
stemmed_data = [stemming(stopword_list) for stopword_list in stopwords]

## Task 3

non-trivial observation 1: How many percent of articles with the word "trump" in it is fake news?

In [11]:
articles_vocabulary = [set(i) for i in tokens]
trump_included = [vocabulary for vocabulary in range(len(articles_vocabulary)) if "trump" in articles_vocabulary[vocabulary]]
trump_fake_news = 0

for i in range(len(trump_included)):
    if datasample['type'][i] == "fake":
        trump_fake_news += 1

print(int(trump_fake_news*100/len(trump_included)),"% of articles where the name 'trump' is present, is a fake news article")

61 % of articles where the name 'trump' is present, is a fake news article


non-trivial observation 2: Is the number of articles spread out tolerably evenly between the domains?

non-trivial observation 3: Is there a link between which domain an article comes from and if it is fake news?

In [13]:
domainList = datasample['domain']
typeList = datasample['type']
domains = set(domainList)

fakeDomainScore = np.zeros(len(domains))
totalDomainScore = np.zeros(len(domains)) 

for i in range (len(domainList)):
    if (typeList[i] == 'fake'):
        index = 0 
        for domain in domains:
            if  domainList[i] == domain:
                fakeDomainScore[index] += 1
            index+=1
    index = 0 
    for domain in domains:
        if  domainList[i] == domain:
            totalDomainScore[index] += 1
        index+=1

print("Each of the 29 domains present in the corpus has the following amount of articles in the corpus:\n", totalDomainScore)
print("\nEach of the 29 domains present in the corpus has the following amount of fake news articles:\n", fakeDomainScore)
print("\nThis means that Beforeitsnews.com has", int(totalDomainScore[np.where(fakeDomainScore == 155)]),"of the articles in the corpus and", int(fakeDomainScore[np.where(fakeDomainScore == 155)]*100/sum(totalDomainScore)), "% of all articles. Thus, the number of articles in the corpus are very unevenly spreed between the domains")
print("\nAlso,", int(fakeDomainScore[np.where(fakeDomainScore == 155)]*100/totalDomainScore[np.where(fakeDomainScore == 155)]), "% of Beforeitsnews.com's articles are fake news and no other domain has fake news in its articles. Thus, there is a link between which domain an article comes from and if it is fake news")

Each of the 29 domains present in the corpus has the following amount of articles in the corpus:
 [  1.   1.   1.   1.   2.   5.   1.   2.   4.   1.   1.   1.   4.  17.
   1.   2.   6.   7.   1.   1.   2. 155.  24.   1.   1.   3.   2.   1.
   1.]

Each of the 29 domains present in the corpus has the following amount of fake news articles:
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0. 155.   0.   0.   0.   0.   0.   0.
   0.]

This means that Beforeitsnews.com has 155 of the articles in the corpus and 62 % of all articles. Thus, the number of articles in the corpus are very unevenly spreed between the domains

Also, 100 % of Beforeitsnews.com's articles are fake news and no other domain has fake news in its articles. Thus, there is a link between which domain an article comes from and if it is fake news


non-trivial observation 4: How many articles have missing author value? 

non-trivial observation 5: How much does missing author increase the likelihood that an article is fake news? 

In [15]:
authors = datasample["authors"]
no_author_counter = 0
no_author_fake_news = 0
no_author_total = 0
author_fake_news = 0
author_total = 0

for i in range(len(authors)):
    if not type(authors[i]) == str:
        no_author_counter += 1
        if datasample["type"][i] == "fake":
            no_author_fake_news += 1
        no_author_total += 1
    elif datasample["type"][i] == "fake":
        author_fake_news += 1
        author_total += 1
    else: 
        author_total += 1

print(int(no_author_counter*100/len(authors)), "% of the articles does not have an author")
print(int(no_author_fake_news*100/no_author_total),'% of the no-author articles are fake news')
print(int(author_fake_news*100/author_total),'% of the articles are fake news')
print('\nThus we see, that having no author on an article only adds two percent points to the likelihood of it being fake')

32 % of the articles does not have an author
63 % of the no-author articles are fake news
61 % of the articles are fake news

Thus we see, that having no author on an article only adds two percent points to the likelihood of it being fake


## Task 4

We have the following 'article start letters'

In [16]:
group_nr = 1
article_start_letters = "ABCDEFGHIJKLMNOPRSTUVWZABCDEFGHIJKLMNOPRSTUVWZ"[group_nr%23:group_nr%23+10]
print(article_start_letters)

BCDEFGHIJK


In [17]:
#The algorithm stops finding new articles when 'stop_searching' is set to True
stop_searching = False

#Finding the nextpage link in the first iteration is a little different, and therefore this value is needed
first_iteration = True

#The root url is the domain of wikinews
root_link = 'https://en.wikinews.org'

#next_page is the webpage that the algorithm searches for articles in next iteration of the while-loop
next_page = root_link + '/w/index.php?title=Category:Politics_and_conflicts'

#The links to the articles starting with the 'article_start_letters' are appended to 'article links'
article_links = []

#For each iteration this list gets some values if the first letter 
#of the first article in the next webpage is between A and K
first_letter_between_B_K = []

#A regex used for 'first_letter_between_B_K'
continue_iterations = re.compile(r"pagefrom=[A-K]")

In [18]:
while not(stop_searching):
    soup = getSoup(next_page)
    articles = soup.find(id="mw-pages")
    
    links = [link.get("href") for link in articles.find_all('a')]
    
    if first_iteration:
        first_letter_between_B_K = continue_iterations.findall(links[0])
        first_iteration = False
        next_page = root_link + links[0]
        article_links += [root_link + group_link for group_link in links[1:] if group_link[6] in article_start_letters]
    else:
        first_letter_between_B_K = continue_iterations.findall(links[1])
        next_page = root_link + links[1]
        article_links += [root_link + group_link for group_link in links[2:] if group_link[6] in article_start_letters]
    
    if len(first_letter_between_B_K) == 0:
        stop_searching = True
    
    first_letter_between_B_K = []

The following box takes some time to run...

In [ ]:
article_source_code = [getSoup(article) for article in article_links]

In [ ]:
article_id = range(1,len(article_links))
article_titles = [article.find('h1').get_text() for article in article_source_code]
article_release_date = [str(article.find(id="publishDate"))[50:60] for article in article_source_code]
article_urls = article_links
article_content = [" ".join([p.get_text() for p in ((article.find(id="mw-content-text")).find(class="mw-parser-output")).findall('p')]) for article in article_source_code]
article_sources = [", ".join([element.get('href') for element in ((article.find('ul')).find_all('a', rel = 'nofollow', class_ ='external text'))]) for article in article_source_code]

In [ ]:
pd_articles = pd.DataFrame()

pd_articles['Id'] = article_id
pd_articles['Titles'] = article_titles
pd_articles['Release_Date'] = article_release_date
pd_articles['url'] = article_urls
pd_articles['content'] = article_content
pd_articles['Sources']= article_sources

pd_articles